<a href="https://colab.research.google.com/github/EyaBey/Testtech/blob/main/Test_technique_EFFIOS_Expertise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Question 1 :Ecrire un code python permettant de lister toutes les combinaisons possibles associées à 23 étapes correspondant à des variables logiques (VRAI/FAUX) puis  d'insérer le résultat dans une table appelée “Combinaisons”

import itertools
import sqlite3

etapes = ["Étape_" + str(i) for i in range(1, 24)]

combinaisons = list(itertools.product([True, False], repeat=23))

conn = sqlite3.connect('ma_base_de_données.db')
cursor = conn.cursor()

cursor.execute('''CREATE TABLE IF NOT EXISTS Combinaisons (ID INTEGER PRIMARY KEY, {})'''.format(", ".join(etapes)))

for i, combinaison in enumerate(combinaisons, 1):
    values = [int(step) for step in combinaison]
    cursor.execute('''INSERT INTO Combinaisons ({}) VALUES ({})'''.format(", ".join(etapes), ", ".join(["?" for _ in etapes])), values)

conn.commit()
conn.close()


In [ ]:
#ici juste pour verifier que les données sont correctement inserées
import sqlite3

conn = sqlite3.connect('ma_base_de_données.db')
cursor = conn.cursor()

cursor.execute('''SELECT COUNT(*) FROM Combinaisons''')
result = cursor.fetchone()[0]
conn.close()

if result == len(combinaisons):
    print("Les combinaisons ont été correctement insérées dans la table.")
else:
    print("Il y a une erreur dans l'insertion des combinaisons dans la table.")

Les combinaisons ont été correctement insérées dans la table.


In [ ]:
#Question 2 :Traduire les règles de gestion métier (des statuts et des sous-statuts) en écriture logique compréhensible par Python et les insérer dans une table appelée “Regles”
import sqlite3

regles = {
    "Statut": {
        "Information avant accrochage": lambda etapes: etapes[0] and etapes[1] and not etapes[2],
        "En cours d'accrochage": lambda etapes: (etapes[0] and etapes[1]) or etapes[2],
        "En production": lambda etapes: etapes[3],
        "Stoppé": lambda etapes: not etapes[0] and not etapes[1] and not etapes[2] and not etapes[3]
    },
    "Sous-Statut": {
        "En cours d'accrochage - Initialisation du projet d'accrochage": lambda etapes: etapes[0] and not etapes[1] and not etapes[2],
        "En cours d'accrochage - Accrochage technique": lambda etapes: etapes[0] and etapes[1] and not etapes[2],
        "En cours d'accrochage - Mise en production": lambda etapes: etapes[0] and etapes[1] and etapes[2],
        "En production - OK": lambda etapes: etapes[3] and not etapes[4] and not etapes[5],
        "En production - Adaptations spécifiques": lambda etapes: etapes[3] and etapes[4] and not etapes[5],
        "En production - Requalification CA": lambda etapes: etapes[3] and not etapes[4] and etapes[5]
    }
}

conn = sqlite3.connect('ma_base_de_données.db')
cursor = conn.cursor()

cursor.execute('''CREATE TABLE IF NOT EXISTS Regles (ID INTEGER PRIMARY KEY, Type TEXT, Nom TEXT, Regle TEXT)''')

for type_statut, regles_statut in regles.items():
    for nom, regle in regles_statut.items():
        cursor.execute('''INSERT INTO Regles (Type, Nom, Regle) VALUES (?, ?, ?)''', (type_statut, nom, regle.__doc__))

conn.commit()
conn.close()


In [ ]:
#Question 3 :Ecrire un code permettant de vérifier des correspondances "combinaison-regle" ...

import sqlite3

def validate_combination(combination, rule):
    if rule is not None:
        return rule(combination)
    else:
        return False

conn = sqlite3.connect('ma_base_de_données.db')
cursor = conn.cursor()

cursor.execute('''CREATE TABLE IF NOT EXISTS correspondance_combinaison_regle (CombinaisonID INTEGER, RegleID INTEGER)''')

cursor.execute('''SELECT * FROM Combinaisons''')
combinations = cursor.fetchall()
cursor.execute('''SELECT * FROM Regles''')
rules = cursor.fetchall()

for combination in combinations:
    for rule in rules:
        if validate_combination(combination[1:], rule[3]):
            cursor.execute('''INSERT INTO correspondance_combinaison_regle (CombinaisonID, RegleID) VALUES (?, ?)''', (combination[0], rule[0]))

conn.commit()
conn.close()
